## Importerer bibliotek

Her importeres nødvendige bibliotek. 

In [ ]:
import requests
import pandas as pd
import numpy as np
import io
from datetime import datetime

from pyspark.sql.types import *
from pyspark.sql import SQLContext

from ssb_sparktools.editing import editing as stedit
from ssb_sparktools.editing import controls as stctrl

## Parametere

Riktig KLASS-tabell velges ved å sette inn rett tabellnummer. Deretter velges dato. (Her velges dagens dato).

In [ ]:
#485
klasstabell=484 
klassdato = datetime.now().strftime("%Y-%m-%d")

## Leser inn KLASS tabell i json, xml og csv format

KLASS-tabellen kan leses inn i ulike formater. Her gis eksempler på å lese den inn som json-, csv- eller xml-fil.

In [ ]:
response_json = requests.get(f'https://data.ssb.no/api/klass/v1/classifications/{klasstabell}/codesAt.json?date={klassdato}').json()

In [ ]:
response_csv = requests.get(f'https://data.ssb.no/api/klass/v1/classifications/{klasstabell}/codesAt.csv?date={klassdato}&csvSeparator=;'\
                            , headers={'accept':'text/csv','charset':'UTF-8'}).text

In [ ]:
response_xml = requests.get(f'https://data.ssb.no/api/klass/v1/classifications/{klasstabell}/codesAt?date={klassdato}')

## KLASS json til pandas

En KLASS-json-fil kan gjøres om til en Pandas-dataframe:

In [ ]:
pd_response_json = pd.DataFrame(response_json['codes'])
pd_response_json

## KLASS csv to pandas

En KLASS-csv kan gjøres om til en Pands-dataframe:

In [ ]:
pd_response_csv = pd.read_csv(filepath_or_buffer = io.StringIO(response_csv), sep=';')
pd_response_csv

## KLASS json til spark dataframe

Slik kan man gjøre om en KLASS-json-fil til en Spark-dataframe:

In [ ]:
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

kodeliste_variabler = [StructField('code', StringType(), True),\
                   StructField('parentCode', StringType(), True),\
                   StructField('level', StringType(), True),\
                   StructField('name', StringType(), True),\
                   StructField('shortName', StringType(), True),\
                  StructField('presentationName', StringType(), True)]
kodeliste_schema = StructType(kodeliste_variabler)

rdd_kodeliste = sc.parallelize(response_json['codes'])
df_kodeliste = sqlContext.createDataFrame(rdd_kodeliste, kodeliste_schema)
df_kodeliste.show(20)

## KLASS csv til spark dataframe

Slik kan man gjøre om en KLASS-csv-fil til en Spark-dataframe (<b> spark_csv</b>, som vil benyttes i eksempelet under):

In [ ]:
csvData = sc.parallelize(response_csv.split("\n"))
spark_csv = spark.read.csv(csvData, sep=';', header=True)
spark_csv.show(20)

## KLASS xml til spark dataframe

Slik kan man gjøre om en KLASS-xml-fil til en Spark-dataframe:

In [ ]:
xmlData = sc.parallelize(response_xml.text.split("\n"))
spark_xml = spark.read.csv(xmlData, sep=',', header=True, multiLine=True)
spark_xml.show(200)

## Lager testdata

Her lages testdata <b>df_spark</b> til bruk i eksemplene under:

In [ ]:
data = [{'ptype': 'alderspensjonFraFolketrygden', 'ant': 2}, {'ptype':'krigspensjon', 'ant': 20}, {'ptype':'feilkode', 'ant': 200}]
df = pd.DataFrame(data)
df_spark = spark.createDataFrame(df)
df_spark.show()

## Kodelistesjekk

Eksempel på sjekk av koder: lager først en liste med alle kodene fra csv-filen <b>df_kodeliste</b>. Deretter brukes funksjonen listcode_check til å sjekke kodene i variabelen <b>ptype</b> i dataframen <b>df_spark</b> opp mot kodene i <b>df_kodeliste</b>. Denne gir beskjed dersom ikke alle kodene ligger i kodelista (den boolske verdien <b>checkres</b> blir "False" dersom det finnes en eller flere koder som ikke matcher med kodelista). Dersom en ønsker å se hvilke(n) kode(r) som ikke ligger på kodelista, kan man printe ut dataframen <b>df_res</b>, som under: 

In [ ]:
df_kodeliste = []
for row in spark_csv.rdd.collect():
    df_kodeliste.append(row['code'])

checkres, df_res  = stctrl.listcode_check(df_spark, 'ptype', df_kodeliste)
if not (checkres):
    print('Ikke alle kodene var i kodelisten, sjekk returnert dataframe for detaljer')

In [ ]:
df_res.show()

## Kodelisteoppslag

Her brukes funksjonen <b>listcode_lookup</b> fra SSB_Spark_tools til å legge på navnet på den aktuelle koden fra kodelista.

In [ ]:
df_lp = stedit.listcode_lookup(df_spark, 'ptype', spark_csv, ['code', 'name'])
df_lp.show(10,False)